In [ ]:
#@title On Time and PPM model (press Play to run)
Extras  = False
import pdb
#Draft last saved 03 Mar 6:22pm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from subprocess import check_output
from datetime import time

_df = pd.read_csv(r"https://raw.githubusercontent.com/C-CA/keras-demos/main/ppm-on-time-new-data.csv")
#C:\Users\Tfarhy\OneDrive - Network Rail\Profile\Desktop\ppm-on-time-new-data.csv
#https://raw.githubusercontent.com/C-CA/keras-demos/main/ppm-on-time-new-data.csv

input_columns = ['Primary Delay per 100 miles','Footfall','Count of Trains/Timing Points - WTT','Planned']
output_columns = ['On Time WTT%','PPM%']

def train():
    global df, features, labels, labelname, input_columns, output_columns, all_columns, model, model2, cot, X, y, X_train, X_test, y_train, y_test, scaler
    
    df = _df.copy()
    df = df[['Primary Delay per 100 miles','Footfall','Count of Trains/Timing Points - WTT','Planned','On Time WTT%','PPM%']]
    
    labelname = buttons.value
    all_columns = input_columns + [labelname]
    
    features = df[input_columns]
    labels = df[labelname]

    X = features.values
    y = labels.values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Scale the data to be between -1 and 1
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Establish model
    model = RandomForestRegressor(n_jobs=-1, verbose = 0)
    model2 = GradientBoostingRegressor(verbose = 0)

    #Make single linear model just for the Count of Trains
    from sklearn.linear_model import LinearRegression
    Xs = df['Planned'].values.reshape(-1,1)
    cot = LinearRegression()
    cot.fit(X = Xs, y = df['Count of Trains/Timing Points - WTT'].values)
        
    model.set_params(n_estimators=70, max_depth = 3)
    model.fit(X_train, y_train)
    model2.fit(X_train, y_train)
    
'''
Cross-validate our model using k-fold cross validation.
'''
def train_and_validate(b):
    global X, y, mae
    print('Generating model...')
    train()
    from sklearn.model_selection import RepeatedKFold
    from sklearn.model_selection import cross_val_score
    from numpy import absolute
    from numpy import mean
    from numpy import std
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, verbose = 0)
    n_scores = absolute(n_scores)
    mae = mean(n_scores)
    print(f'Containing variables {list(df.columns)} \nTargeting {labelname} as output')
    print(f'Cross-validated mean absolute error for model: {mae*100:.2f}%, standard deviation {std(n_scores):.2f}')
    
    inference_widgets()
    trend(None)

def infer(b, printout = True, whichmodel =['RF','GBM']):
    global cotvalue
    prediction = np.nan
    prediction2 = np.nan
    
    inference_vector = [box.value for box in boxes]
    cotvalue = int(cot.predict(np.array(PlannedBox.value, ndmin = 2))[0])
    inference_vector.insert(2, cotvalue)
    raw_inference = inference_vector.copy()
    inference_vector = scaler.transform(np.array(inference_vector, ndmin = 2))
    
    if 'RF' in whichmodel:
        prediction = model.predict(inference_vector)[0]
    if 'GBM' in whichmodel:
        prediction2 = model2.predict(inference_vector)[0]
    if printout:
        matrix.value = str([round(inference,2) for inference in raw_inference])
        pred_display.value = f'Predicted a {labelname} of {prediction*100:.2f}%.'
    else:
        return {'RF':prediction, 'GBM':prediction2 }
    

def SliderChanged(change):
    
    inference = infer(None, printout = False, whichmodel = 'GBM')
    inferbutton.description = f"Predict {labelname} [live estimate = {inference['GBM']*100:.2f}%]"

    cotlabel.value = f'Count of Trains/Timing Points - WTT = {cotvalue}'
    
    FootfallPercentage.value = f'{round(FootfallBox.value/25790, 1)}%'
    PlannedPercentage.value = f'{round(PlannedBox.value/241, 1)}%'
def splom(b, color_continuous_scale = 'plasma'):
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    plot_df = _df.copy()
    plot_df = plot_df.loc[plot_df['Primary Delay per 100 miles']<2]
    
    plot_columns = all_columns.copy()
    plot_columns.remove('Primary Delay per 100 miles')
    fig = px.scatter_matrix(plot_df,
                            color_continuous_scale = color_continuous_scale,
                            dimensions = plot_columns,
                            color = 'Primary Delay per 100 miles',
                           title = 'Correlations',
                            hover_data=['Date'],
                           opacity = 1,
                           )    
    fig.update_layout(height = 1000,)
    fig.update_traces(diagonal_visible=False)
    fig.show()
    
def trend(b):
    #FootfallPercentage.value = f'{round(FootfallBox.value/25790, 1)}%'
    #PlannedPercentage.value = f'{round(PlannedBox.value/241, 1)}%'
    import plotly.express as px

    ffmax = 2579000
    plannedmax  = 24100

    #labelname = 'On Time WTT%'
    trendmodel = RandomForestRegressor(n_jobs=-1, verbose = 0)
    trendx = df[['Primary Delay per 100 miles','Footfall','Planned']]
    trendy = df[labelname]

    trendmodel.set_params(n_estimators=70,)
    trendmodel.fit(trendx.values, trendy.values)

    mesh = np.mgrid[0:5:0.2, ffmax*.3:ffmax*.4+1:ffmax*.05, 8000:23001:300].reshape(3, -1).T
    trendpred = trendmodel.predict(mesh)

    trenddf = pd.DataFrame(mesh, columns = trendx.columns)
    trenddf[labelname] = trendpred

    trenddf['Footfall'] = trenddf['Footfall']*100/ffmax
    trenddf['Footfall'] = trenddf['Footfall'].astype(int)
    trenddf['Footfall'] = trenddf['Footfall'].astype(str)
    trenddf['Footfall'] = trenddf['Footfall'] + '%'
    trenddf[labelname] = trenddf[labelname]*100
    
    if labelname == 'PPM%':
        range_y = [70,100]
    else:
        range_y = [65,90]

    fig = px.scatter(trenddf, x="Planned", y=labelname, animation_frame='Primary Delay per 100 miles', color="Footfall", trendline="lowess",
                     width = 1000, height = 700, range_x=[7000,25000], range_y = range_y )

    fig["layout"].pop("updatemenus") # optional, drop animation buttons

    fig.show()
#Jupyter widgets

from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout, Label, HBox, VBox
label_layout = Layout(width='500px',height='21px', font_size = '50px')
def inference_widgets():
    global hboxes, sliders, boxes, vbox, pred_display, matrix, inferbutton, FootfallBox, PlannedBox, PrimaryDelayBox,FootfallSlider, PlannedSlider, PrimaryDelaySlider, FootfallPercentage,PlannedPercentage, PlannedLabel, cotlabel
 

    input_variable = 'Primary Delay per 100 miles'
    PrimaryDelaySlider = widgets.FloatSlider(min = 0, max = 12, value = int(df[58:59][input_variable]), layout=Layout(width='200px'), 
                                             readout = False,  continuous_update=True, step = 0.2)
    PrimaryDelayBox = widgets.BoundedFloatText(value = int(df[58:59][input_variable]), layout=Layout(width='120px', padding = '0px 10px 0px 10px'), step = 0.2,  readout_format='.3f')
    PrimaryDelayHBox = HBox([PrimaryDelaySlider,
                      PrimaryDelayBox,
                      Label(input_variable),])

    
    input_variable = 'Footfall'
    FootfallSlider = widgets.IntSlider(min = 100000, max = 4000000, value = int(df[58:59][input_variable]), layout=Layout(width='200px'), 
                                       readout = False,  continuous_update=True)
    FootfallBox = widgets.IntText(value = int(df[58:59][input_variable]), layout=Layout(width='120px', padding = '0px 10px 0px 10px')  )
    FootfallPercentage = Label(f'')
    FootfallHBox =HBox([FootfallSlider, 
             FootfallBox,
             Label(input_variable),
             FootfallPercentage,
            ])

    
    input_variable = 'Planned'
    PlannedSlider = widgets.IntSlider(min = 8000, max = 30000, value = int(df[58:59][input_variable]), layout=Layout(width='200px'), 
                                      readout = False,  continuous_update=True)
    PlannedBox = widgets.IntText(value = int(df[58:59][input_variable]), layout=Layout(width='120px', padding = '0px 10px 0px 10px')  )
    PlannedLabel  = Label(input_variable)
    PlannedPercentage = Label(f'')
    PlannedHBox =HBox([PlannedSlider, 
             PlannedBox,
             PlannedLabel,
             PlannedPercentage,
            ])
    
    
    cotlabel = Label(f'Count of Trains/Timing Points - WTT = {int(df[58:59]["Count of Trains/Timing Points - WTT"].values[0])}')
    
    hboxes =    [PrimaryDelayHBox, FootfallHBox, PlannedHBox]
    sliders =   [PrimaryDelaySlider, FootfallSlider, PlannedSlider]
    boxes =     [PrimaryDelayBox, FootfallBox, PlannedBox]
    
    for slider, box in zip(sliders, boxes):
        widgets.link((box, 'value'), (slider, 'value'))
        box.observe(SliderChanged, 'value')
    
    vbox = VBox(hboxes+[cotlabel])
    
    matrix = Label(f'',layout=label_layout)
    pred_display = Label(f'',layout=label_layout)
    inferbutton = widgets.Button(description=f"Predict {labelname}", layout=Layout(width='300px'))
    inferbutton.on_click(infer)
    corr = widgets.Button(description="Show correlations")
    corr.on_click(splom)
    
    display(vbox)
    display(inferbutton)
    display(corr)
    display(matrix)
    display(pred_display)
buttons = widgets.ToggleButtons(
    options=output_columns,
    description='Predict what:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or '',
#     icons=['check'] * 3
)

generate = widgets.Button(description="Generate model")
generate.on_click(train_and_validate)

display(buttons)
display(generate)

#train_and_validate(None)
